# Installation

Before exploring the notebook you need to clone the main repository:
```bash
 git clone https://github.com/kalmarek/1812.03456.git
```
This notebook should be located in `1812.03456/notebooks` directory.

In the main directory (`1812.03456`) you should run the following code in `julia`s `REPL` console to instantiate the environment for computations:
```julia
using Pkg
Pkg.activate(".")
Pkg.instantiate()
```
(this needs to be done once per installation).

Instantiation should install (among others): the [`SCS` solver][1], [`JuMP` package][2] for mathematical programming and `IntervalArithmetic.jl` package from [`ValidatedNumerics.jl`][3].

The environment also makes use of [`AbstractAlgebra.jl`][4], [`Nemo.jl`][5], [`GroupRings.jl`][6], [`Groups.jl`][7] and [`PropertyT.jl`][8] packages.

[1]: https://github.com/cvxgrp/scs  
[2]: https://github.com/JuliaOpt/JuMP.jl  
[3]: https://github.com/JuliaIntervals/ValidatedNumerics.jl
[4]: https://github.com/Nemocas/AbstractAlgebra.jl
[5]: https://github.com/Nemocas/Nemo.jl
[6]: https://github.com/kalmarek/GroupRings.jl
[7]: https://github.com/kalmarek/Groups.jl
[8]: https://github.com/kalmarek/PropertyT.jl

# The computation

The following programme certifies that
$$\operatorname{Adj}_4 + \operatorname{Op}_4 - 0.82\Delta_4 =\Sigma_i \xi_i^*\xi_i \in \Sigma^2_2\mathbb{R}\operatorname{SL}(4,\mathbb{Z}).$$

With small changes (which we will indicate) it also certifies that 
$$\operatorname{Adj}_3 - 0.157999\Delta_3 \in \Sigma^2_2\mathbb{R}\operatorname{SL}(3,\mathbb{Z})$$
and that
$$\operatorname{Adj}_5 +1.5 \mathrm{Op}_5 - 1.5\Delta_5 \in \Sigma^2_2\mathbb{R}\operatorname{SL}(5,\mathbb{Z}).$$

In [2]:
using Pkg
Pkg.activate("..")
using Dates
now()

2019-06-28T10:13:53.177

In [3]:
using AbstractAlgebra
using Nemo
using Groups
using GroupRings
using PropertyT


Welcome to Nemo version 0.14.0

Nemo comes with absolutely no warranty whatsoever



┌ Info: Recompiling stale cache file /home/kalmar/.julia/compiled/v1.1/PropertyT/ASnhZ.ji for PropertyT [03b72c93-0167-51e2-8a1e-eb4ff1fb940d]
└ @ Base loading.jl:1184


So far we only made the needed packages available in the notebook. 
In the next cell we define `G` to be the set of all $4\times 4$ matrices over $\mathbb Z$.
(For the second computation, set `N=3` below; for the third, set `N=5`)

In [4]:
N = 4
G = Nemo.MatrixSpace(Nemo.ZZ, N, N)

Matrix Space of 4 rows and 4 columns over Integer Ring

## Generating set
Now we create the elementary matrices $E_{i,j}$. The set of all such matrices and their inverses is denoted by `S`.

In [5]:
include("../src/sqadjop.jl")

SqAdjOp (generic function with 1 method)

In [7]:
S = generating_set(G)

24-element Array{fmpz_mat,1}:
 [1 1 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1] 
 [1 0 1 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1] 
 [1 0 0 1]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1] 
 [1 0 0 0]
[1 1 0 0]
[0 0 1 0]
[0 0 0 1] 
 [1 0 0 0]
[0 1 1 0]
[0 0 1 0]
[0 0 0 1] 
 [1 0 0 0]
[0 1 0 1]
[0 0 1 0]
[0 0 0 1] 
 [1 0 0 0]
[0 1 0 0]
[1 0 1 0]
[0 0 0 1] 
 [1 0 0 0]
[0 1 0 0]
[0 1 1 0]
[0 0 0 1] 
 [1 0 0 0]
[0 1 0 0]
[0 0 1 1]
[0 0 0 1] 
 [1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[1 0 0 1] 
 [1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 1 0 1] 
 [1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 1 1] 
 [1 -1 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1]
 [1 0 -1 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1]
 [1 0 0 -1]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1]
 [1 0 0 0]
[-1 1 0 0]
[0 0 1 0]
[0 0 0 1]
 [1 0 0 0]
[0 1 -1 0]
[0 0 1 0]
[0 0 0 1]
 [1 0 0 0]
[0 1 0 -1]
[0 0 1 0]
[0 0 0 1]
 [1 0 0 0]
[0 1 0 0]
[-1 0 1 0]
[0 0 0 1]
 [1 0 0 0]
[0 1 0 0]
[0 -1 1 0]
[0 0 0 1]
 [1 0 0 0]
[0 1 0 0]
[0 0 1 -1]
[0 0 0 1]
 [1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[-1 0 0 1]
 [1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 -1 0 1]
 [1 

## Group Ring and Laplacians
Now we will generate the ball `E_R` of radius $R=4$ in $\operatorname{SL}(N,\mathbb{Z})$ and use this as a (partial) basis in a group ring (denoted by `RG` below). Such group ring also needs a multiplication table (`pm`, which is actually a *division table*) which is created as follows: when $x,y$ reside at positions `i`-th and `j`-th in `E_R`, then `pm[i,j] = k`, where `k` is the position of $x^{-1}y$ in `E_R`. 

In [8]:
ID = one(G)
radius = 2
E_R, sizes = Groups.generate_balls(S, ID, radius=2*radius);
E_rdict = GroupRings.reverse_dict(E_R)
pm = GroupRings.create_pm(E_R, E_rdict, sizes[radius]; twisted=true);
RG = GroupRing(G, E_R, E_rdict, pm)
@show sizes;
Δ = length(S)*one(RG) - RG(S)

sizes = [25, 433, 6149, 75197]


24[1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1] - 1[1 1 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1] - 1[1 0 1 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1] - 1[1 0 0 1]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1] - 1[1 0 0 0]
[1 1 0 0]
[0 0 1 0]
[0 0 0 1] - 1[1 0 0 0]
[0 1 1 0]
[0 0 1 0]
[0 0 0 1] - 1[1 0 0 0]
[0 1 0 1]
[0 0 1 0]
[0 0 0 1] - 1[1 0 0 0]
[0 1 0 0]
[1 0 1 0]
[0 0 0 1] - 1[1 0 0 0]
[0 1 0 0]
[0 1 1 0]
[0 0 0 1] - 1[1 0 0 0]
[0 1 0 0]
[0 0 1 1]
[0 0 0 1] - 1[1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[1 0 0 1] - 1[1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 1 0 1] - 1[1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 1 1] - 1[1 -1 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1] - 1[1 0 -1 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1] - 1[1 0 0 -1]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1] - 1[1 0 0 0]
[-1 1 0 0]
[0 0 1 0]
[0 0 0 1] - 1[1 0 0 0]
[0 1 -1 0]
[0 0 1 0]
[0 0 0 1] - 1[1 0 0 0]
[0 1 0 -1]
[0 0 1 0]
[0 0 0 1] - 1[1 0 0 0]
[0 1 0 0]
[-1 0 1 0]
[0 0 0 1] - 1[1 0 0 0]
[0 1 0 0]
[0 -1 1 0]
[0 0 0 1] - 1[1 0 0 0]
[0 1 0 0]
[0 0 1 -1]
[0 0 0 1] - 1[1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[-1 0 0 1] - 

## Orbit Decomposition
Now something happens: in the next cell we split the subspace of $\mathbb{R} \operatorname{SL}(N, \mathbb{Z})$ supported on `E_R` into irreducible representations of the wreath product $\mathbb Z / 2 \mathbb Z \wr \operatorname{Sym}_N$. The action of wreath product on the elements of the matrix space is by conjugation, i.e. permutation of rows and columns.
We also compute projections on the invariant subspaces to later speed up the optimisation step.

In [52]:
od = PropertyT.OrbitData(RG, WreathProduct(PermGroup(2), PermGroup(N)))
orbit_data = PropertyT.decimate(od);

┌ Info: Decomposing basis of RG into orbits of
│   autS = Wreath Product of Permutation group over 2 elements by Permutation group over 4 elements
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/has1D/src/orbitdata.jl:15


  0.466664 seconds (7.08 M allocations: 137.645 MiB)
  0.044372 seconds (294.02 k allocations: 13.655 MiB)


┌ Info: The action has 558 orbits
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/has1D/src/orbitdata.jl:18
┌ Info: Finding projections in the Group Ring of
│   autS = Wreath Product of Permutation group over 2 elements by Permutation group over 4 elements
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/has1D/src/orbitdata.jl:20
┌ Info: Finding AutS-action matrix representation
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/has1D/src/orbitdata.jl:23


  3.146962 seconds (6.21 M allocations: 120.969 MiB, 52.03% gc time)
  0.025616 seconds (5.96 k allocations: 13.453 MiB)


┌ Info: Computing the projection matrices Uπs
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/has1D/src/orbitdata.jl:27


  1.093775 seconds (61.50 k allocations: 756.110 MiB)


┌ Info: 
│ multiplicities  =   3  13  19  12  10   0   0   0   9  11  13  15   0   0   0   1   1   1   2   1
│     dimensions  =   1   3   3   2   1   4   8   4   6   6   6   6   4   8   4   1   3   3   2   1
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/has1D/src/orbitdata.jl:37
┌ Info: Sparsifying (433, 3)-matrix... 
│  0.6882217090069284   → 0.18475750577367206 ), (240 non-zeros)
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/has1D/src/orbitdata.jl:114
┌ Info: Sparsifying (433, 13)-matrix... 
│  0.5821637946349263   → 0.3084029134837449  ), (1736 non-zeros)
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/has1D/src/orbitdata.jl:114
┌ Info: Sparsifying (433, 19)-matrix... 
│  0.6031360155585268   → 0.4934970220007293  ), (4060 non-zeros)
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/has1D/src/orbitdata.jl:114
┌ Info: Sparsifying (433, 12)-matrix... 
│  0.6043110084680523   → 0.2817551963048499  ), (1464 non-zeros)
└ @ PropertyT /home/kalmar/.julia/packages/Pro

## Elements Adj and Op
Now we  define the elements $\operatorname{Adj}_N$ and $\operatorname{Op}_N$. The functions `Sq`, `Adj`, `Op` returning the appropriate elements are defined in the `src/sqadjop.jl` source file.

In [45]:
@time AdjN = Adj(RG, N)
@time OpN = Op(RG, N);

  0.253026 seconds (4.23 k allocations: 645.141 KiB)
  0.050150 seconds (3.27 k allocations: 170.953 KiB)


Finally we compute the element `elt` of our interest:
* if `N=3`: $\operatorname{elt} = \operatorname{Adj}_3$
* if `N=4`: $\operatorname{elt} = \operatorname{Adj}_4 + \operatorname{Op}_4$
* if `N=5`: $\operatorname{elt} = \operatorname{Adj}_5 + 1.5\operatorname{Op}_5.$

In [46]:
if N == 3
    k = 0
elseif N == 4
    k = 1
elseif N == 5
    k = 1.5
end
elt = AdjN + k*OpN;
elt.coeffs

75197-element SparseArrays.SparseVector{Int64,Int64} with 361 stored entries:
  [1    ]  =  488
  [2    ]  =  -40
  [3    ]  =  -40
  [4    ]  =  -40
  [5    ]  =  -40
  [6    ]  =  -40
  [7    ]  =  -40
  [8    ]  =  -40
  [9    ]  =  -40
  [10   ]  =  -40
           ⋮
  [418  ]  =  1
  [420  ]  =  1
  [422  ]  =  2
  [423  ]  =  1
  [424  ]  =  1
  [425  ]  =  1
  [426  ]  =  1
  [428  ]  =  1
  [429  ]  =  1
  [430  ]  =  1
  [431  ]  =  1

## Optimization Problem

We are ready to define the optimisation problem. Function
> `PropertyT.SOS_problem(x, Δ, orbit_data; upper_bound=UB)`  

defines the optimisation problem equivalent to the one of the form
\begin{align}
\text{ maximize : } \quad & \lambda\\
\text{under constraints : }\quad & 0 \leqslant \lambda \leqslant \operatorname{UB},\\
     & x - \lambda \Delta = \sum \xi_i^* \xi_i,\\
     & \text{each $\xi_i$ is invariant under $\mathbb{Z}/2\mathbb{Z} \wr \operatorname{Sym}_N$}.
\end{align}

In [47]:
RG.basis[34]

[1 1 0 0]
[0 1 0 0]
[0 0 1 1]
[0 0 0 1]

In [48]:
orbit_data.orbits[9]

48-element Array{Int64,1}:
  34
 201
  45
 279
 100
 204
 111
 327
  73
 126
  84
 306
  53
   ⋮
 121
 132
 224
 342
 103
 258
 114
 330
  37
 255
  48
 282

In [51]:
print(OpN.coeffs[orbit_data.orbits[9]])

  [1 ]  =  2
  [2 ]  =  2
  [3 ]  =  2
  [4 ]  =  2
  [5 ]  =  2
  [6 ]  =  2
  [7 ]  =  2
  [8 ]  =  2
  [9 ]  =  1
  [10]  =  1
  [11]  =  1
  [12]  =  1
  [13]  =  1
  [14]  =  1
  [15]  =  1
  [16]  =  1
  [17]  =  1
  [18]  =  1
  [19]  =  1
  [20]  =  1
  [21]  =  1
  [22]  =  1
  [23]  =  1
  [24]  =  1
  [25]  =  1
  [26]  =  1
  [27]  =  1
  [28]  =  1
  [29]  =  1
  [30]  =  1
  [31]  =  1
  [32]  =  1
  [33]  =  1
  [34]  =  1
  [35]  =  1
  [36]  =  1
  [37]  =  1
  [38]  =  1
  [39]  =  1
  [40]  =  1
  [41]  =  2
  [42]  =  2
  [43]  =  2
  [44]  =  2
  [45]  =  2
  [46]  =  2
  [47]  =  2
  [48]  =  2

In [12]:
# @time SDP_problem, varλ, varP = PropertyT.SOS_problem(elt, Δ, orbit_data)
if N == 3
    UB = 0.158
elseif N == 4
    UB = 0.82005
elseif N == 5
    UB = 1.5005
end
SDP_problem, varP = PropertyT.SOS_problem(elt, Δ, orbit_data; upper_bound=UB)

┌ Info: Adding 558 constraints...
└ @ PropertyT /home/kalmar/.julia/packages/PropertyT/has1D/src/sos_sdps.jl:92


AssertionError: AssertionError: all(k .== val)

In [11]:
using JuMP
using SCS
λ = Ps = warm = nothing

### Solving the problem
Depending on the actual problem one may need to tweak the parameters given to the solver:
 * `eps` sets the requested accuracy
 * `max_iters` sets the number of iterations to run before solver gives up
 * `alpha` is a parameter ($\alpha \in (0,2)$) which determines the rate of convergence at the cost of the accuracy
 * `acceleration_lookback` should not be changed, unless you know what are the consequences ;-)
 
 The parameters below should be enough to obtain a decent solution for $\operatorname{SL}(4, \mathbb{Z}), \operatorname{SL}(5, \mathbb{Z})$.   
 For $\operatorname{SL}(3, \mathbb{Z})$ approximately `1_000_000` of iterations is required; in this case by changing `UB` to $0.15$ (above) a much faster convergence can be observed.

In [12]:
with_SCS = with_optimizer(SCS.Optimizer, 
    linear_solver=SCS.Direct, 
    eps=3e-13,
    max_iters=30000,
    alpha=1.5,
    acceleration_lookback=1,
    warm_start=true)

status, warm = PropertyT.solve(SDP_problem, with_SCS, warm);

----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 131318
eps = 3.00e-13, alpha = 1.50, max_iters = 30000, normalize = 1, scale = 1.00
acceleration_lookback = 1, rho_x = 1.00e-03
Variables n = 1388, constraints m = 1946
Cones:	primal zero / dual free vars: 1196
	linear vars: 1
	sd vars: 749, sd blks: 14
Setup time: 9.65e-02s
SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.02e+00  1.23e+00  9.75e-01 -3.44e+01  4.76e+00  0.00e+00  3.24e-03 
   100| 2.29e-05  3.30e-04  4.55e-04 -8.19e-01 -8.18e-01  1.65e-15  2.86e-01 
   200| 3.14e-05  1.93

In [13]:
λ = value(SDP_problem[:λ])
Ps = [value.(P) for P in varP]
@show(status, λ);

status = OPTIMAL::TerminationStatusCode = 1
λ = 0.8200499999995887


## Checking the solution
Now we reconstruct the solution to the original problem over $\mathbb{R} \operatorname{SL}(N,\mathbb{Z})$, which essentially boils down to averaging the obtained solution over the wreath product.

In [14]:
Qs = real.(sqrt.(Ps));
Q = PropertyT.reconstruct(Qs, orbit_data);

As explained in the paper the columns of the square-root of the solution matrix provide the coefficients for $\xi_i$'s in basis `E_R` of the group ring. Below we compute the residual 
    $$ b = \left(x - \lambda\Delta\right) - \sum \xi_i^*\xi_i.$$
As we do it in floating-point arithmetic,  the result can't be taken seriously.

In [15]:
function SOS_residual(x::GroupRingElem, Q::Matrix)
    RG = parent(x)
    @time sos = PropertyT.compute_SOS(RG, Q);
    return x - sos
end

SOS_residual (generic function with 1 method)

In [16]:
b = SOS_residual(elt - λ*Δ, Q)
@show norm(b, 1);

┌ Warning: Scalar and coeffs are in different rings! Promoting result to Float64
└ @ GroupRings /home/kalmar/.julia/packages/GroupRings/RGQkD/src/GroupRings.jl:300
┌ Warning: Adding elements with different coefficient rings, Promoting result to Float64
└ @ GroupRings /home/kalmar/.julia/packages/GroupRings/RGQkD/src/GroupRings.jl:338


  0.056232 seconds (977 allocations: 2.065 MiB)
norm(b, 1) = 1.4587019788678534e-8


### Checking in interval arithmetic

In [17]:
using IntervalArithmetic
IntervalArithmetic.setrounding(Interval, :tight)
IntervalArithmetic.setformat(sigfigs=12);

Here we resort to interval arithmetic to provide certified upper and lower bounds on the norm of the residual.
* We first change entries of `Q` to narrow intervals
* We project columns of `Q` so that $0$ is in the sum of coefficients of each column (i.e. $\xi_i \in I \operatorname{SL}(N,\mathbb{Z})$)
* We compute the sum of squares and the $\ell_1$-norm of the residual in the interval arithmetic.

The returned `check_columns_augmentation` is a boolean flag to detect if the projection was successful, i.e. if we can guarantee that each column of `Q_aug` can be represented by an element from the augmentation ideal. (If it were not successful, one may project `Q = PropertyT.augIdproj(Q)` in the floating point arithmetic prior to the cell below).

The resulting norm of the residual is **guaranteed** to be contained in the resulting interval. E.g. if each entry of `Q` were changed into an honest rational number and all the computations were carried out in rational arithmetic, the rational $\ell_1$-norm will be contained in the interval $\ell_1$-norm.

In [18]:
Q_aug, check_columns_augmentation = PropertyT.augIdproj(Interval, Q);
@assert check_columns_augmentation
elt_int = elt - @interval(λ)*Δ;
b_int = SOS_residual(elt_int, Q_aug)
@show norm(b_int, 1);

┌ Warning: Scalar and coeffs are in different rings! Promoting result to Interval{Float64}
└ @ GroupRings /home/kalmar/.julia/packages/GroupRings/RGQkD/src/GroupRings.jl:300
┌ Warning: Adding elements with different coefficient rings, Promoting result to Interval{Float64}
└ @ GroupRings /home/kalmar/.julia/packages/GroupRings/RGQkD/src/GroupRings.jl:338


  4.050721 seconds (976 allocations: 4.070 MiB)
norm(b_int, 1) = [1.82538570099e-08, 1.85760547024e-08]


In [19]:
certified_λ = @interval(λ) - 2^2*norm(b_int,1)

[0.820049925695, 0.820049926985]

So $\operatorname{elt} - \lambda_0 \Delta \in \Sigma^2 I\operatorname{SL}(N, \mathbb{Z})$, where as $\lambda_0$ we could take the left end of the above interval:

In [20]:
certified_λ.lo

0.8200499256953697

In [21]:
now()

2019-06-27T00:46:54.431

In [22]:
versioninfo()

Julia Version 1.1.1
Commit 55e36cc308 (2019-05-16 04:10 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i5-6200U CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)
Environment:
  JULIA_NUM_THREADS = 2
